In [1]:
print("hello world")

hello world


In [4]:
# ## 1. 라이브러리 설치 및 로그인

# Octo 모델 및 TRL 최신 버전 설치
%pip install -U -q git+https://github.com/huggingface/trl.git transformers datasets accelerate peft trl bitsandbytes einops

%pip install pandas

%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu129
# %pip install hf_xet

%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu129
Note: you may need to restart the kernel to use updated packages.
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Hugging Face Hub 로그인
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
# ## 2. 라이브러리 임포트 및 기본 설정

import torch
from transformers import AutoModel, AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer
from datasets import Dataset
from PIL import Image


In [4]:
# ## 3. 데이터셋 준비
#
# **중요! 수정된 부분**
#
# - `Assets_Zara01` 폴더의 CSV 파일과 이미지 데이터를 로드하도록 수정되었습니다.
# - 각 CSV 파일은 하나의 에피소드(trajectory)를 나타냅니다.
# - 각 샘플은 (현재 이미지, 자연어 명령, 현재 액션)으로 구성됩니다.
# - **자연어 명령을 CSV의 시작/목표 위치를 사용하여 동적으로 생성하도록 변경했습니다.**
# - 액션은 `[Action_X, Action_Z]` 2차원 벡터를 사용합니다.

import os
import pandas as pd
import glob

# 현재 작업 디렉토리 (프로젝트 루트)
project_root = os.getcwd()
data_root = os.path.join(project_root, "Assets_Zara01")
state_files_path = os.path.join(data_root, "Zara01_State")

# 모든 CSV 파일 경로 가져오기
csv_files = glob.glob(os.path.join(state_files_path, "*.csv"))

raw_dataset = []

# 각 CSV 파일(에피소드)을 순회
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    
    if df.empty:
        continue
        
    # 에피소드의 시작 위치와 목표 위치 추출 (첫 번째 행 기준)
    start_pos_row = df.iloc[0]
    start_pos = [start_pos_row['Position_X'], start_pos_row['Position_Y'], start_pos_row['Position_Z']]
    goal_pos = [start_pos_row['GoalPosition_X'], start_pos_row['GoalPosition_Y'], start_pos_row['GoalPosition_Z']]
    
    # 영어로 자연어 명령 생성 (소수점 2자리까지)
    instruction = (
        f"From the start position [{start_pos[0]:.2f}, {start_pos[1]:.2f}, {start_pos[2]:.2f}], "
        f"reach the goal position [{goal_pos[0]:.2f}, {goal_pos[1]:.2f}, {goal_pos[2]:.2f}]."
    )
    
    # 에피소드의 각 타임스텝을 순회하며 데이터 샘플 생성
    for index, row in df.iterrows():
        # CSV에 있는 이미지 경로는 'Assets/...'로 시작하는 상대 경로입니다.
        # os.path.abspath를 사용하여 현재 작업 디렉토리 기준으로 절대 경로를 생성합니다.
        relative_image_path = row['FovImagePath'].replace('/', os.sep).replace('Assets', 'Assets_Zara01')
        image_path = os.path.abspath(relative_image_path)

        # 액션 데이터 추출
        action = [row['Action_X'], row['Action_Z']]
        
        # 데이터 샘플 생성
        raw_dataset.append({
            "observation_images": [image_path],
            "instruction": instruction, 
            "action": action,
        })

# 데이터셋 크기 확인 (너무 많으면 일부만 사용)
print(f"Total samples created: {len(raw_dataset)}")
# 예시로 처음 5개 샘플 출력
print("Example samples:")
for i in range(min(5, len(raw_dataset))):
    print(raw_dataset[i])

# Hugging Face Dataset 객체로 변환
# 전체 데이터가 너무 클 경우, 메모리 부족을 방지하기 위해 일부만 사용할 수 있습니다.
# 예: hf_dataset = Dataset.from_list(raw_dataset[:1000])
hf_dataset = Dataset.from_list(raw_dataset)

Total samples created: 50172
Example samples:
{'observation_images': ['/home/rlawlsgus/github/VLMFinetuningToy/Assets_Zara01/FoVImages/zara01_146/t_0.jpg'], 'instruction': 'From the start position [17.03, 0.00, 2.57], reach the goal position [-1.32, 0.00, 2.60].', 'action': [-143.94, 1.8039]}
{'observation_images': ['/home/rlawlsgus/github/VLMFinetuningToy/Assets_Zara01/FoVImages/zara01_146/t_1.jpg'], 'instruction': 'From the start position [17.03, 0.00, 2.57], reach the goal position [-1.32, 0.00, 2.60].', 'action': [-143.94, 1.8039]}
{'observation_images': ['/home/rlawlsgus/github/VLMFinetuningToy/Assets_Zara01/FoVImages/zara01_146/t_2.jpg'], 'instruction': 'From the start position [17.03, 0.00, 2.57], reach the goal position [-1.32, 0.00, 2.60].', 'action': [-143.94, 1.8029]}
{'observation_images': ['/home/rlawlsgus/github/VLMFinetuningToy/Assets_Zara01/FoVImages/zara01_146/t_3.jpg'], 'instruction': 'From the start position [17.03, 0.00, 2.57], reach the goal position [-1.32, 0.00, 

In [ ]:
# ## 4. 모델 로드
# - BitsAndBytesConfig를 사용하여 4비트 양자화로 모델을 로드하여 메모리를 절약합니다.
# - `AutoModelForVision2Seq`는 지원 중단 예정이므로 권장되는 `AutoModelForImageTextToText`를 사용합니다.
# - `trust_remote_code=True`를 설정하여 Hugging Face Hub의 모델 코드를 직접 실행하도록 허용합니다.

from transformers import AutoModelForImageTextToText

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 사용할 Octo 모델 ID
model_id = "rail-berkeley/octo-small-1.5"

# 모델과 프로세서 로드
model = AutoModel.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True  # model_type 오류 해결
)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)


ValueError: Unrecognized model in rail-berkeley/octo-small-1.5. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_anything, depth_pro, d

In [ ]:
# ## 5. 데이터 포맷팅 함수 정의
# 
# **중요! 수정이 필요한 부분**
# 
# - `SFTTrainer`에 데이터를 올바르게 전달하기 위한 함수입니다.
# - Octo는 `text`와 `images`를 입력으로 받습니다. `text`는 `processor.tokenizer.apply_chat_template`을 사용하여 생성합니다.
# - `action`을 정규화하고 토큰화하는 과정이 필요할 수 있습니다. 이 부분은 Octo의 공식 예제를 참고하여 데이터에 맞게 조정해야 합니다.

def format_for_octo(sample):
    # 1. 이미지 로드
    images = [Image.open(path).convert("RGB") for path in sample["observation_images"]]
    
    # 2. 텍스트(명령)를 대화 템플릿에 맞게 변환
    # Octo는 특정 대화 형식을 따릅니다.
    text = processor.tokenizer.apply_chat_template(
        [{"role": "user", "content": f"<image>\n{sample['instruction']}"}],
        tokenize=False,
        add_generation_prompt=True
    )
    
    # 3. 모델 입력 생성
    inputs = processor(text=text, images=images, return_tensors="pt")
    
    # 4. 레이블(정답 액션) 처리
    # 중요! 액션 값을 모델 출력에 맞게 변환해야 합니다.
    # Octo는 액션을 이산적인 토큰으로 예측하므로, 연속적인 액션 값을 binning(양동이질)하는 과정이 필요합니다.
    # 아래는 간단한 예시이며, 실제로는 데이터의 분포를 보고 bin 경계를 정해야 합니다.
    action_bins = torch.linspace(-1.0, 1.0, 256) # -1에서 1 사이를 256개 구간으로 나눔
    
    # numpy 배열을 torch 텐서로 변환
    action_tensor = torch.tensor(sample["action"], dtype=torch.float32)
    
    # 각 액션 차원 값을 가장 가까운 bin의 인덱스(토큰 ID)로 변환
    # 이 부분이 Octo 학습의 핵심입니다!
    action_labels = torch.bucketize(action_tensor, action_bins)
    
    # SFTTrainer는 labels가 input_ids와 같은 길이를 기대하는 경우가 많습니다.
    # 여기서는 입력 텍스트 부분은 무시하고(-100), 액션 부분만 학습하도록 레이블을 생성합니다.
    input_len = inputs["input_ids"].shape[1]
    labels = torch.full((1, input_len + len(action_labels)), -100)
    labels[0, input_len:] = action_labels
    
    # input_ids와 attention_mask에 액션 레이블을 이어붙입니다.
    # 이 부분은 모델 아키텍처나 학습 방식에 따라 달라질 수 있습니다.
    # 가장 간단한 방식은 SFTTrainer가 텍스트 생성처럼 처리하도록 하는 것입니다.
    # 이 경우, 액션 토큰을 텍스트의 일부처럼 이어붙여야 합니다.
    
    # 여기서는 간단하게 input_ids와 labels만 반환하고,
    # SFTTrainer의 data_collator가 처리하도록 위임하는 방식을 가정합니다.
    # 더 정확한 구현을 위해서는 Octo의 공식 학습 스크립트를 참고하는 것이 좋습니다.
    
    # 이 스켈레톤에서는 SFTTrainer에 직접 딕셔너리를 전달하기 위해
    # 데이터셋 자체를 변환하는 방식을 사용합니다.
    
    return {
        "text": text,
        "images": images,
        "labels": action_labels.tolist() # 리스트로 변환하여 데이터셋에 저장
    }

# 데이터셋 변환 (이 방식은 메모리를 많이 사용할 수 있습니다. 큰 데이터셋은 map을 사용하세요)
# processed_dataset = hf_dataset.map(format_for_octo, remove_columns=list(hf_dataset.features))



In [ ]:
# ## 6. PEFT (LoRA) 설정
# - 기존 코드와 유사하게 LoRA를 설정하여 효율적인 파인튜닝을 합니다.

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="VISION_2_SEQ_MODEL", # Octo와 같은 모델은 이 타입을 사용
    target_modules=["q_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# get_peft_model 함수는 더 이상 명시적으로 필요하지 않을 수 있습니다.
# Trainer가 내부적으로 처리합니다.


In [ ]:
# ## 7. SFTTrainer 설정 및 학습
# - `SFTConfig`를 사용하여 학습 파라미터를 정의합니다.
# - `SFTTrainer`에 모델, 데이터셋, PEFT 설정 등을 전달하여 학습을 시작합니다.

training_args = SFTConfig(
    output_dir="octo-small-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=2, # GPU 메모리에 따라 조절
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    learning_rate=1e-5,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="no", # 평가 데이터셋이 없다면 no
    bf16=True,
    # --- SFTTrainer를 위한 추가 인수 ---
    dataset_text_field="text", # 데이터셋에서 텍스트 필드를 지정
    max_seq_length=1024, # 최대 시퀀스 길이
    # packing=True, # 여러 짧은 샘플을 묶어 효율성 증대
)

# SFTTrainer 초기화
# 중요!: SFTTrainer는 기본적으로 텍스트 데이터를 처리합니다.
# Vision-Language 모델을 위해서는 커스텀 collator를 제공하거나,
# TRL의 최신 기능이 processor를 통해 이미지 처리를 지원하는지 확인해야 합니다.
# 아래 코드는 processor를 직접 trainer에 전달하여 이미지 처리를 위임하는 방식입니다.

# TRL이 processor를 내부적으로 사용하여 이미지를 텐서로 변환하도록 함
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset, # 전처리되지 않은 원본 데이터셋 전달
    dataset_map_function=format_for_octo, # map 함수를 트레이너에 위임 (메모리 효율적)
    peft_config=peft_config,
    processor=processor, # Processor를 전달
)


# 학습 시작
trainer.train()

# 모델 저장
trainer.save_model("./octo-small-finetuned/final_checkpoint")